In [2]:
cd ../../../..

/Users/jbbutler129/Google Drive (butlerj@berkeley.edu)/Classes/215A/final_project/rule-vetting


In [3]:
%pwd

'/Users/jbbutler129/Google Drive (butlerj@berkeley.edu)/Classes/215A/final_project/rule-vetting'

# Preprocessing Steps 

This notebook describes the list of steps taken in preprocessing the dataset, with information/plots to justify certain decisions and judgement calls. The steps here mirror the steps in the preprocess_data function of dataset.py. We also provide information on potential perturbations to the judgment calls, where appropriate. Note that, when there are multiple perturbations to a judgement call that can be made, we will opt to pursue the default perturbations listed in dataset.py, for consistency.

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from rulevetting.projects.tbi_pecarn.helper import union_var

In [5]:
# useful helper functions

def make_missing_plot(df):
    
    col_dict = {}

    for col in list(df):
    
        col_dict[col] = sum(np.isnan(np.array(df[col])))/len(df)

    col_names = list(col_dict.keys())
    proportions = list(col_dict.values())

    name = 'nan_props.png'

    plt.figure(dpi=800, figsize=(17, 7))
    plt.bar(col_names, proportions)
    plt.ylabel('proportion', fontsize = 16)
    plt.xticks(rotation = 45, ha = 'right', fontsize = 10)
    plt.title('Proportion of NaN for each Variable (no response)', fontsize = 20)
    
def make_missing_contingency(df, varlist):

    subset = df[varlist]
    subset = subset.fillna('missing')
    
    first_col = subset[varlist[0]]
    other_cols = []
    
    for col in list(subset.drop(varlist[0], axis = 1)):
        other_cols.append(subset[col])

    table = pd.crosstab(first_col, other_cols, dropna = False)
    
    return table



In [6]:
tbi_df = pd.read_csv("data/tbi_pecarn/raw/TBI PUD 10-08-2013.csv")

### Step 1: Remove variables which have nothing to do with our problem (uncontroversial choices)


#### List of Variables:

List 1:

*EmplType*
*Certification*

Justification: probably not a huge systematic difference between who fills out the form. This would be an issue if there were systematic differences that we would also expect in future data, but we can't tell. Domain expert also says not a big deal, and many of the questions are fairly objective anyway, so these variables are unimportant

List 2:

*InjuryMech*

Justification: a 10 way categorical variable would probably not be super helpful for a future data situation, maybe just keep the severity of the injury mechanism (although generalizability to a future data situation might be tough, but I think the paper uses this variable soo)

List 3:

*CTForm1*
*IndAge*
*IndAmnesia*
*IndAMS*
*IndClinSFx*
*IndHA*
*IndHema*
*IndLOC*
*IndMech*
*IndNeuroD*
*IndRqstMD*
*IndRqstParent*
*IndRqstTrauma*
*IndSeiz*
*IndVomit*
*IndXraySFx*
*IndOth*
*CTSed*
*CTSedAgitate*
*CTSedAge*
*CTSedRqst*
*CTSedOth*

Justification: these variables all represent details relevant to obtaining a CT scan for an individual (reasons why, whether they were sedated, etc.) This information is not relevant for our problem, so we just drop.

List 4:

*AgeTwoPlus*

Justification: AgeTwoPlus is just some variable that we could make on our own. Would be useful to eventually subset observations by this, but maybe not entirely necessary, so for the purposes of having a clean dataset with everything in it (before making judgment calls to subset for different models), we'll just get rid of this for now (less to worry about in cleaning/EDA)

List 5:

*Observed*
*EDDisposition*
*CTDone*
*EDCT*
*PosCT*
*Finding1*
...
*Finding23*

Justification: this is all info on what was found in the CT scans/what ended up being done with the patient (operating room, admitted into hospital, etc). All of this information would be known on future data well after our algorithm is to be used, so we exclude this information.

In [7]:
list1 = ['EmplType', 'Certification']
list2 = ['InjuryMech']

# grab all of the CT/Ind variables
list3 = []
for col in tbi_df.keys():
    if 'Ind' in col or 'CT' in col:
        list3.append(col)

list4 = ['AgeTwoPlus', 'AgeInMonth']

# grab all of the Finding variables
list5 = ['Observed', 'EDDisposition']

for col in tbi_df.keys():
    if 'Finding' in col:
        list5.append(col)

total_rem = list1 + list2 + list3 + list4 + list5

tbi_df_step1 = tbi_df.drop(total_rem, axis=1)

### Step 2: Remove variables with really high missingness (that we don't care about)

In [8]:
make_missing_plot(tbi_df_step1)

Seems like Dizzy and Ethnicity have very high rates of missingness. Dr. Inglis said Dizzy was too subjective, so probably not a useful predictor, and we would only keep ethnicity around to do some post-hoc analysis, but if so much of the ethnicity variable is missing, we wouldn't be able to tell if our classifier preferentially detects ciTBI in certain ethnicities. So, we will just drop this as well. Not worth dropping observations or imputing for missing values of these columns

In [10]:
tbi_df_step2 = tbi_df_step1.drop(['Ethnicity', 'Dizzy'], axis = 1)

### Step 3: Remove observations with GCS < 14


Justification: the paper does this as well, so as to be best comparable to the baseline results from the paper. Also, Rob told us GCS < 14 means you're pretty wacked out, and we're interested in predicting on children in a borderline case (for which we could possibly avoid a CT scan). Also, drop the GCSGroup column once we're done because we could just remake it with the other GCS info.

In [11]:
tbi_df_step3 = tbi_df_step2[tbi_df_step2['GCSGroup'] == 2]
tbi_df_step3 = tbi_df_step3.drop(['GCSGroup'], axis = 1)

### Step 4: Generate a Unified Response Variable (?)

We try to improve upon the outcome variable from the original dataset (PosIntFinal)

In [27]:
sum(tbi_df_step3['PosIntFinal'].isnull())

18

There are only 18 observations with missing PosIntFinal outcomes. The paper simply gets rid of these, but the number of missing observations for the other four outcome variables is each less than 18, meaning that there are some values we can impute. Looking at the following 5-way contingency table, we see that 15 of the 18 missing observations in the outcome have all the possible outcome variables as 0, so we could impute in these cases.

In [20]:
make_missing_contingency(tbi_df_step3, ['PosIntFinal', 'HospHead', 'DeathTBI', 'Intub24Head', 'Neurosurgery'])

HospHead        0.0                                              ... missing  \
DeathTBI        0.0                         missing              ...     0.0   
Intub24Head     0.0     1.0     missing         0.0     1.0      ...     1.0   
Neurosurgery    0.0 1.0 0.0 1.0     0.0 1.0     0.0 1.0 0.0 1.0  ...     0.0   
PosIntFinal                                                      ...           
0.0           41934   0   0   0       2   0       4   0   0   0  ...       0   
1.0               0   2   0   0       0   0       0   0   0   0  ...       0   
missing          15   0   0   0       0   0       1   0   0   0  ...       0   

HospHead                                                      
DeathTBI                     missing                          
Intub24Head      missing         0.0     1.0     missing      
Neurosurgery 1.0     0.0 1.0     0.0 1.0 0.0 1.0     0.0 1.0  
PosIntFinal                                                   
0.0            0       0   0       0   0   0   0       0   0  
1.0            0       0   0       0   0   0   0       0   0  
missing        0       0   0       0   0   0   0       0   0  

[3 rows x 36 columns]

In [15]:
make_missing_contingency(tbi_df_step3, ['PosIntFinal', 'DeathTBI'])

DeathTBI,0.0,missing
PosIntFinal,,
0.0,42032,4
1.0,376,0
missing,16,2


In [16]:
make_missing_contingency(tbi_df_step3, ['PosIntFinal', 'Intub24Head'])

Intub24Head,0.0,1.0,missing
PosIntFinal,,,
0.0,42034,0,2
1.0,368,8,0
missing,17,0,1


In [17]:
make_missing_contingency(tbi_df_step3, ['PosIntFinal', 'Neurosurgery'])

Neurosurgery,0.0,1.0
PosIntFinal,,
0.0,42036,0
1.0,316,60
missing,18,0


In [28]:
make_missing_contingency(tbi_df_step3, ['PosIntFinal', 'HospHead'])

HospHead,0.0,1.0,missing
PosIntFinal,,,
0.0,41940,95,1
1.0,2,374,0
missing,17,0,1


However, this mismatch with the missing values of PosIntFinal and known values for all of the categories on which PosIntFinal is based is slightly concerning. So, we opt to make a new outcome variable which is the union of all of the four outcome variables provided, plus the PosIntFinal variable. By union, we mean that if any of the outcomes are 1, we indicate this observation as having a ciTBI. Otherwise, this is not a ciTBI (no evidence of a ciTBI). In this way, we combine all of the information available to get a more complete outcome variable.

In [33]:
tbi_df_step4 = union_var(tbi_df_step3, ['DeathTBI', 'Intub24Head', 'Neurosurgery', 'HospHead', 'PosIntFinal'], "ciTBI")


In [38]:
sum(tbi_df_step4['ciTBI'].isnull())

0

There are no missing values in the outcome anymore!

### Step 5: Impute/drop GCS Verbal/Motor/Eye Scores

In [39]:
tbi_df_step5 = tbi_df_step4.copy()
make_missing_contingency(tbi_df_step5[tbi_df_step5['GCSTotal'] == 14], ['GCSVerbal', 'GCSMotor', 'GCSEye'])


GCSMotor  5.0              6.0               missing            
GCSEye    3.0 4.0 missing  3.0   4.0 missing     3.0 4.0 missing
GCSVerbal                                                       
4.0         0   0       0    0  1028       0       0   2       1
5.0         0  41       0  251     1       0       0   0       0
missing     0   0       0    0     1       0       0   0      19

Seems like of the GCS = 14 scores, only 19 of them are missing all three, and about 20 observations are missing at least one value. 

(JUDGEMENT CALL) But this is out of 1200 observations, so we can just get rid of these, probably not going to hurt us.

In [41]:

tbi_df_step5.drop(tbi_df_step5[(tbi_df_step5['GCSTotal'] == 14) & ((tbi_df_step5['GCSVerbal'].isnull()) | (tbi_df_step5['GCSMotor'].isnull()) | (tbi_df_step5['GCSEye'].isnull()))].index, inplace = True)


Now, look at observations whose GCS scores are 15

In [42]:
make_missing_contingency(tbi_df_step5[tbi_df_step5['GCSTotal'] == 15], ['GCSVerbal', 'GCSMotor', 'GCSEye'])

GCSMotor  4.0             6.0                missing            
GCSEye    3.0 4.0 missing 3.0    4.0 missing     3.0 4.0 missing
GCSVerbal                                                       
4.0         0   0       0   0      0       2       0   0       1
5.0         0   0       0   0  39820      10       0  15       0
missing     0   0       1   1      7       0       0   6    1223

Then, impute the missing values among GCS = 15 scores to just be the full points for each category.

In [43]:
tbi_df_step5.loc[(tbi_df_step5['GCSTotal'] == 15) & tbi_df_step5['GCSVerbal'].isnull(), 'GCSVerbal'] = 5
tbi_df_step5.loc[(tbi_df_step5['GCSTotal'] == 15) & tbi_df_step5['GCSMotor'].isnull(), 'GCSMotor'] = 6
tbi_df_step5.loc[(tbi_df_step5['GCSTotal'] == 15) & tbi_df_step5['GCSEye'].isnull(), 'GCSEye'] = 4

Now, make the contingency table among the GCS = 15 observations again

In [44]:
make_missing_contingency(tbi_df_step5[tbi_df_step5['GCSTotal'] == 15], ['GCSVerbal', 'GCSMotor', 'GCSEye'])

GCSMotor  4.0     6.0       
GCSEye    3.0 4.0 3.0    4.0
GCSVerbal                   
4.0         0   0   0      3
5.0         0   1   1  41081

Also seems like there were a few wonky observations (like GCS score was 15, but they had lower than optimal scores in some of the individual categories, so what I will do is just drop those observations). JUDGEMENT CALL

In [45]:
tbi_df_step5.drop(tbi_df_step5[(tbi_df_step5['GCSTotal'] == 15) & ((tbi_df_step5['GCSVerbal'] < 5) | (tbi_df_step5['GCSMotor'] < 6) | (tbi_df_step5['GCSEye'] < 4))].index, inplace = True)


One final check for observations with GCS = 14:

In [46]:
make_missing_contingency(tbi_df_step5[tbi_df_step5['GCSTotal'] == 14], ['GCSVerbal', 'GCSMotor', 'GCSEye'])


GCSMotor  5.0      6.0      
GCSEye    3.0 4.0  3.0   4.0
GCSVerbal                   
4.0         0   0    0  1028
5.0         0  41  251     1

That seems to have mostly solved the issues, but there is still one big issue: one observation has full 15 score among the three categories, but is labelled as 14. Let's drop it. (JUDGEMENT CALL)

In [47]:
tbi_df_step5.drop(tbi_df_step5[(tbi_df_step5['GCSTotal'] == 14) & ((tbi_df_step5['GCSVerbal'] == 5) & (tbi_df_step5['GCSMotor'] == 6) & (tbi_df_step5['GCSEye'] == 4))].index, inplace = True)


### Step 6: Impute/drop based on Paralyzed/Sedated/Intubated

Let's make a contingency table of missingness for these variables:

In [48]:
make_missing_contingency(tbi_df_step5, ['Sedated', 'Paralyzed', 'Intubated'])

Paralyzed    0.0             1.0             missing            
Intubated    0.0 1.0 missing 0.0 1.0 missing     0.0 1.0 missing
Sedated                                                         
0.0        42044   0       0   0   0       0       4   0       0
1.0           28   0       0   0   4       0       0   0       0
missing        5   0       0   0   0       0       6   0     310

Looks like a lot of missing values. Let's just get rid of any observation that has a 1 in it (a little too intense for our population; we don't care about people who were sedated, paralyzed, or intubated). This is according to Dr. Inglis, who said any individual experiencing any of these variables is probably not in the borderline case where we would could see if a CT scan could be avoided.

Also remove the observations that have missing values in any of these columns because we can't particularly tell whether they were paralyzed, sedated, or not. We should be careful not to accidentally include observations that were paralyzed, sedated, or intubated since this would skew our population away from the target population, so this is why we opt to remove any observation with amiguity in these categories.

In [49]:
tbi_df_step6 = tbi_df_step5.copy()

tbi_df_step6.drop(tbi_df_step6.loc[(tbi_df_step6['Paralyzed'] == 1) | (tbi_df_step6['Sedated'] == 1) | (tbi_df_step6['Intubated'] == 1)].index, inplace = True)
tbi_df_step6.drop(tbi_df_step6.loc[(tbi_df_step6['Paralyzed'].isnull()) | (tbi_df_step6['Sedated'].isnull()) | (tbi_df_step6['Intubated'].isnull())].index, inplace = True)
tbi_df_step6.drop(['Sedated', 'Paralyzed', 'Intubated'], axis = 1, inplace = True)


### Step 7: Impute/drop based on AMS

In [51]:
tbi_df_step7 = tbi_df_step6.copy()

make_missing_contingency(tbi_df_step7, ['AMS', 'AMSAgitated', 'AMSSleep', 'AMSSlow', 'AMSRepeat'])

AMSAgitated   0                                      ... 92                    \
AMSSleep      0                                  1   ... 1  92                  
AMSSlow       0            1         92          0   ... 92 0        1          
AMSRepeat     0    1  92   0   1  92 0  1  92    0   ... 92 0  1  92 0  1  92   
AMS                                                  ...                        
0.0            0    0  0    0   0  0  0  0  0     0  ...  0  0  0  0  0  0  0   
1.0          916  277  0  756  50  0  0  0  0  2152  ...  0  0  0  0  0  0  0   
missing        0    0  0    0   0  0  0  0  0     0  ...  0  0  0  0  0  0  0   

AMSAgitated               
AMSSleep                  
AMSSlow     92            
AMSRepeat   0  1      92  
AMS                       
0.0          0  0  36363  
1.0          0  0      0  
missing      0  0    280  

[3 rows x 81 columns]

Seems like from this that all of the missing AMS observations are actually missing for all of the other variables as well (subvariables, all have label 92 indicating a missing value or no AMS). So, let's drop these observations since AMS is an important predictor and we don't want to bastardize this for our classifier (also only ~300 observations, not the biggest deal; JUDGEMENT CALL). Note that because we have dropped all of the truly missing values, every other '92' observation for the AMS subvariables now corresponds to just having no altered mental state in the first place

In [16]:
tbi_df_step7.drop(tbi_df_step7.loc[tbi_df_step8['AMS'].isnull()].index, inplace = True)

### Step 8: Impute/drop based on OSI

In [56]:
tbi_df_step8 = tbi_df_step7.copy()

make_missing_contingency(tbi_df_step8, ['OSI', 'OSIExtremity', 'OSICut', 'OSICspine', 'OSIFlank', 'OSIAbdomen', 'OSIPelvis'])

OSIExtremity   0                                    ... 92                    \
OSICut         0                                    ... 92                     
OSICspine      0                                    ... 92                     
OSIFlank       0                                1   ... 1  92                  
OSIAbdomen     0           1         92         0   ... 92 0        1          
OSIPelvis      0   1  92   0   1  92 0  1  92   0   ... 92 0  1  92 0  1  92   
OSI                                                 ...                        
0.0             0   0  0    0   0  0  0  0  0    0  ...  0  0  0  0  0  0  0   
1.0           616  72  0  297  11  0  0  0  0  350  ...  0  0  0  0  0  0  0   
missing         0   0  0    0   0  0  0  0  0    0  ...  0  0  0  0  0  0  0   

OSIExtremity               
OSICut                     
OSICspine                  
OSIFlank                   
OSIAbdomen   92            
OSIPelvis    0  1      92  
OSI                        
0.0           0  0  37657  
1.0           0  0      0  
missing       0  0    166  

[3 rows x 729 columns]

Once again, seems like all of the missing values in this column cannot be imputed since these values are also missing all of their subvariable info. So, we drop (only 181 observations, but this is another JUDGMENT CALL since domain expert says this variable should probably not be important at all, alternative to this judgment call could be to just drop the column but keep the obs with missing values anyways, and fit the models without this column and see if the results change). We encode this perturbation in our framework in dataset.py.

In [59]:

tbi_df_step8.drop(tbi_df_step8.loc[tbi_df_step8['OSI'].isnull()].index, inplace = True)

### Step 9: Impute/drop based on Hema variables

In [61]:
tbi_df_step9 = tbi_df_step8.copy()
make_missing_contingency(tbi_df_step9, ['Hema', 'HemaLoc', 'HemaSize'])

HemaLoc    1.0                           2.0                          ...  \
HemaSize   1.0   2.0   3.0 92.0 missing  1.0   2.0  3.0 92.0 missing  ...   
Hema                                                                  ...   
0.0          0     0     0    0       0    0     0    0    0       0  ...   
1.0       1697  5013  1578    0     375  809  2045  548    0      96  ...   
missing      0     0     0    0       0    0     0    0    0       0  ...   

HemaLoc  92.0                        missing                       
HemaSize  1.0 2.0 3.0   92.0 missing     1.0 2.0 3.0 92.0 missing  
Hema                                                               
0.0         0   0   0  25105       0       0   0   0    0       0  
1.0         0   0   0      0       0      37  65  18    0      78  
missing     0   0   0    225       0       0   0   0    0       0  

[3 rows x 25 columns]

There are a few judgment calls we can make here to deal with the missingness:

1. If any of Hema, HemaLoc, or HemaSize have a nonmissing value indicating the presence of any of these variables, the individual had a Hematoma, and we only save this one column
2. Ignore HemaSize (drop it), impute the presence of Hema from HemaLoc
3. Drop any observation that has a missing value from any subcategory (DEFAULT)

And then regardless of any of the above, drop the missing Hema observations.

In [63]:

tbi_df_step9.drop(tbi_df_step9.loc[(tbi_df_step9['Hema'].isnull()) | (tbi_df_step9['HemaLoc'].isnull()) | (tbi_df_step9['HemaSize'].isnull())].index, inplace = True)


### Step 10: Impute/drop based on skull fracture palp variables

JUDGMENT CALL: it seems important to include if you have an unclear exam for a palpable skull fracture (probably will come up a lot in future situations, so important not to subset this out of the population). Since we are trying to predict low risk of clinically important TBI (vs. not low risk) we should probably frame the variable into a binary variable where one choice could correspond to something that would not indicate a TBI (no trauma) vs something that could indicate a TBI (unclear exam or yes trauma). So, turn the 2 into 1s for SFxPalp. The variable means 1 if they did not not observe a fracture, and 0 if they did no observe a fracture.

In [64]:
tbi_df_step10 = tbi_df_step9.copy()
tbi_df_step10.loc[(tbi_df_step10['SFxPalp'] == 2), 'SFxPalp'] = 1

Anterior Fontanelle bulging is a really good indicator according to the domain expert, so we will throw out these observations that are missing this value. Also, seems like most of these missing values are also missing the palpable depressed variable (or it's not applicable). So, our judgment call is that we'll delete this, since the domain expert says this is an important variable and it'll be interesting to see if it's actually important. Also, let's drop the observations that have missing SFxPalpDepress variables.

In [65]:
tbi_df_step10.drop(tbi_df_step10.loc[(tbi_df_step10['FontBulg'].isnull()) | (tbi_df_step10['SFxPalpDepress'].isnull()) | (tbi_df_step10['SFxPalp'].isnull())].index, inplace = True)


### Step 11: Impute/drop based on basilar skull fracture variables

In [67]:
tbi_df_step11 = tbi_df_step10.copy()
varlist = ['SFxBas', 'SFxBasHem', 'SFxBasOto', 'SFxBasPer', 'SFxBasRet', 'SFxBasRhi']
make_missing_contingency(tbi_df_step11, varlist)

SFxBasHem  0                                ... 92                          \
SFxBasOto  0                                ... 92                           
SFxBasPer  0                            1   ... 1  92                        
SFxBasRet  0         1        92        0   ... 92 0        1        92      
SFxBasRhi  0  1  92  0  1  92 0  1  92  0   ... 92 0  1  92 0  1  92 0  1    
SFxBas                                      ...                              
0.0         0  0  0   0  0  0  0  0  0   0  ...  0  0  0  0  0  0  0  0  0   
1.0        15  9  0  23  0  0  0  0  0  66  ...  0  0  0  0  0  0  0  0  0   
missing     0  0  0   0  0  0  0  0  0   0  ...  0  0  0  0  0  0  0  0  0   

SFxBasHem         
SFxBasOto         
SFxBasPer         
SFxBasRet         
SFxBasRhi     92  
SFxBas            
0.0        40041  
1.0            0  
missing      329  

[3 rows x 243 columns]

Now, we analyze the basilar skull fracture variables. It seems like all of the missing basilar skull fracture variables are also missing all of the subvariables, so it is impossible to impute. So, let's just get rid of these observations. JUDGMENT CALL

In [68]:

tbi_df_step11.drop(tbi_df_step11.loc[tbi_df_step11['SFxBas'].isnull()].index, inplace = True)

### Step 12: Impute/drop based on Clav group of variables

In [71]:
tbi_df_step12 = tbi_df_step11.copy()
varlist = ['Clav', 'ClavFace', 'ClavNeck', 'ClavFro', 'ClavOcc', 'ClavPar', 'ClavTem']
make_missing_contingency(tbi_df_step12, varlist)


ClavFace   0                                        ... 92                    \
ClavNeck   0                                        ... 92                     
ClavFro    0                                        ... 92                     
ClavOcc    0                                    1   ... 1  92                  
ClavPar    0             1          92          0   ... 92 0        1          
ClavTem    0    1  92    0    1  92 0  1  92    0   ... 92 0  1  92 0  1  92   
Clav                                                ...                        
0.0         0    0  0     0    0  0  0  0  0     0  ...  0  0  0  0  0  0  0   
1.0       255  789  0  2274  107  0  0  0  0  3277  ...  0  0  0  0  0  0  0   
missing     0    0  0     0    0  0  0  0  0     0  ...  0  0  0  0  0  0  0   

ClavFace               
ClavNeck               
ClavFro                
ClavOcc                
ClavPar  92            
ClavTem  0  1      92  
Clav                   
0.0       0  0  14490  
1.0       0  0      0  
missing   0  0     50  

[3 rows x 729 columns]

Similar to before, any missing observation for the *Clav* variable is actually missing values for all of the subcategories, so impossible to impute and we opt to just drop.

In [73]:
tbi_df_step12.drop(tbi_df_step12.loc[tbi_df_step12['Clav'].isnull()].index, inplace = True)

### Step 13: Impute/drop based on Neuro group of variables

In [74]:
tbi_df_step13 = tbi_df_step12.copy()
varlist = ['NeuroD', 'NeuroDMotor', 'NeuroDSensory', 'NeuroDCranial', 'NeuroDReflex', 'NeuroDOth']
make_missing_contingency(tbi_df_step13, varlist)

NeuroDMotor    0                                 ... 92                       \
NeuroDSensory  0                                 ... 92                        
NeuroDCranial  0                             1   ... 1  92                     
NeuroDReflex   0          1        92        0   ... 92 0        1        92   
NeuroDOth      0    1  92 0  1  92 0  1  92  0   ... 92 0  1  92 0  1  92 0    
NeuroD                                           ...                           
0.0             0    0  0  0  0  0  0  0  0   0  ...  0  0  0  0  0  0  0  0   
1.0            12  251  0  5  2  0  0  0  0  60  ...  0  0  0  0  0  0  0  0   
missing         0    0  0  0  0  0  0  0  0   0  ...  0  0  0  0  0  0  0  0   

NeuroDMotor              
NeuroDSensory            
NeuroDCranial            
NeuroDReflex             
NeuroDOth     1      92  
NeuroD                   
0.0            0  39483  
1.0            0      0  
missing        0    255  

[3 rows x 243 columns]

Similar to before, any missing observation for the *Neuro* variable is actually missing values for all of the subcategories, so impossible to impute and we opt to just drop.

In [75]:
tbi_df_step13.drop(tbi_df_step13.loc[tbi_df_step13['NeuroD'].isnull()].index, inplace = True)

### Step 14: Impute/drop based on Vomiting group of variables


Now, lets deal with the vomiting variables. We can probably get rid of the VomitStart, VomitLast, and Vomit Number, variables, since these don't matter according to the physician (and would not be helpful for interpretability). JUDGEMENT CALL. Then, we should drop observations that are missing the vomiting variables altogether. 

In [77]:
tbi_df_step14 = tbi_df_step13.copy()
tbi_df_step14.drop(['VomitStart', 'VomitLast', 'VomitNbr'], axis = 1, inplace = True)
tbi_df_step14.drop(tbi_df_step14.loc[tbi_df_step14['Vomit'].isnull()].index, inplace = True)


To further justify dropping the number of vomits, notice that all of the missing values are all individuals who did vomit. It wouldn't make sense to impute and then use the variable as a predictor (might be sketchy/skew conclusions) but wouldn't make sense to drop the observations because we would be dropping a big proportion of observations that had vomiting observed (and could skew the dataset). So, let's just get rid of this column. Pretty sure the paper discusses this as a potential predictor, but ends up only using whether the vomit occurred or not. It's entirely possible that vomiting was marked but the number was simply not, so this should be fine.

### Step 15: Impute/drop based on Headache group of variables

In [81]:
tbi_df_step15 = tbi_df_step14.copy()
varlist = ['HA_verb', 'HASeverity']
make_missing_contingency(tbi_df_step15, varlist)

HASeverity,1.0,2.0,3.0,92.0,missing
HA_verb,,,,,
0.0,0,0,0,14918,0
1.0,5030,5356,758,0,919
91.0,0,0,0,12443,0
missing,0,0,0,361,0


For the headache groups of variables, we have a few options for judgement calls/different imputation strategies:

1. Among verbal participants, impute HA_verb by the presence of either HA_verb, HAStart, or HASeverity (when did it start, and how severe), and then drop the features on when it started and how severe since domain knowledge says this is not too important
2. Impute HA_verb by the presence of a severity score (among individuals who are verbal), and then drop headache start since this is known not to be very important
3. (option 1) First drop the column headache start, and any observation that is missing any of the headache group variables (DEFAULT)
3. (option 2) Drop any observation that is missing any of the headache group variables

Then, regardless of the above, drop the missing headache variables

In [82]:
tbi_df_step15.drop(['HAStart'], axis = 1, inplace = True)
tbi_df_step15.drop(tbi_df_step15.loc[(tbi_df_step15['HA_verb'].isnull()) | (tbi_df_step15['HASeverity'].isnull())].index, inplace = True)

### Step 16: Impute/drop based on Seizure group of variables

As before, there are a few imputation strategies that we can adopt, each can be considered a judgement call:

1. If any of Seiz, SeizLen, or SeizOccur are marked as nonmissing and indicate a seizure, we say the individual had a seizure (taking the union of the the columns Seiz, SeizLen, and SeizOccur)
2. Drop the SeizOccur column, impute the presence of Seiz from the presence of a nonmissing SeizLen, while dropping the missing SeizLen
3. (option 1) drop SeizOccur, and drop any observations missing any of Seiz or SeizLen (DEFAULT)
3. (option 2) drop any observation missing any of Seiz, SeizLen, or SeizOccur

Regardless of the above, we will drop the remaining missing Seiz columns

In [83]:
tbi_df_step16 = tbi_df_step15.copy()

In [85]:
varlist = ['Seiz', 'SeizLen']
make_missing_contingency(tbi_df_step16, varlist)

SeizLen,1.0,2.0,3.0,4.0,92.0,missing
Seiz,,,,,,
0.0,0,0,0,0,37523,0
1.0,219,135,20,5,0,67
missing,0,0,0,0,536,0


All of the observations that are missing seizures have missing values for every other variable, so it would be impossible to impute (JUDGMENT CALL, unless we would want to impute somehow else but that's something to think about later). There are 67 observations that are missing the length but did identify seizures, but I'm just going to drop those anyways.

In [88]:
tbi_df_step16.drop(tbi_df_step16.loc[(tbi_df_step16['Seiz'].isnull()) | (tbi_df_step16['SeizLen'].isnull())].index, inplace = True)
tbi_df_step16.drop('SeizOccur', axis = 1, inplace = True)

### Step 17: Impute/drop based on Loss of Consciousness variables

In [92]:
tbi_df_step17 = tbi_df_step16.copy()
varlist = ['LOCSeparate', 'LocLen']
make_missing_contingency(tbi_df_step17, varlist)

LocLen,1.0,2.0,3.0,4.0,92.0,missing
LOCSeparate,,,,,,
0.0,0,0,0,0,31348,0
1.0,487,1220,826,145,0,1043
2.0,343,400,97,23,0,736
missing,0,0,0,0,1234,0


LOCSeparate should be recoded as a binary variable, where 0 represents definitively no loss of consciousness and 1 is the opposite (JUDGEMENT CALL). However, weird that most observations with suspected loss of consciousness nonetheless have lengths/durations of the loss of consciousness. I think the paper groups the binary variable the way we did, so I will do this as well! Otherwise, it's impossible to impute missing loss of consciousness since the lengths are all missing, and similarly, impossible to impute missing values of the length of the loss of consciousness, so we will also get rid of these.

In [91]:
tbi_df_step17 = tbi_df_step16.copy()
tbi_df_step17.drop(tbi_df_step17.loc[(tbi_df_step17['LOCSeparate'].isnull()) | (tbi_df_step17['LocLen'].isnull())].index, inplace = True)
tbi_df_step17.loc[(tbi_df_step17['LOCSeparate'] == 2), 'LOCSeparate'] = 1

### Step 18: Drop Missing Values for Amnesia/High Injury Severity

For the last two variables, (amnesia and high injury severity), we'll just drop all of the missing observations because these would possibly be useful. It's impossible to impute missing values for these observations from the info that we have, and these might be important predictors according to the physician, so it would need this information to be as present as possible.

In [94]:
tbi_df_step18 = tbi_df_step17.copy()
tbi_df_step18.drop(tbi_df_step18.loc[(tbi_df_step18['Amnesia_verb'].isnull()) | (tbi_df_step18['High_impact_InjSev'].isnull())].index, inplace = True)



### Step 19: Drop the Drugs Column

Physician said this would not be extremely important

In [95]:
tbi_df_step19 = tbi_df_step18.drop('Drugs', axis = 1)

### Step 20: Handle ActNorm

There were a significant number of missing values in the ActNorm column (whether the parent thinks the child is acting normally/like themselves). However, Dr. Inglis said he really believes in trying to identify this as a predictor, since even though it is subjective, there is something to the parents' opinion of how the child is acting to determine if a ciTBI is present. So, we wish to impute this as much as possible. There are two obvious ways to impute these values:

1. Assume that a missing value in this column indicates the child is acting normally (assumption: if the child is acting normally, the parent might skip over this question altogether, not answer it). (DEFAULT)
2. Assume that a missing value in this column indicates the child is acting abnormally (assumption: one of the options on the form is 'unknown'. We do not know if a missing value here is marked 'unknown' or not, but assuming all of them are 'unknown', we will take this to be the child is acting abnormally, since if there is even a question that the child is acting like themself, they are probably not acting like themselves. So, the interpretation of the variable, under this assumption, becomes '1' if definitively acting like themselves, '0' otherwise). 

### Write to CSV

In [31]:
#tbi_df_step20.to_csv('data/tbi_pecarn/cleaned/clean_dataset_11_24.csv', index=False)

## Concluding remarks

So, at this point, we've gotten rid of most of the missing values. I've left the missing values for Race and ActNorm, because these seem particularly important and may warrant some further investigation to decide what we should do. Otherwise, I still haven't dealt with the missing values that were labelled as 92, but after looking through the missing values so much in contingency tables, it seems like all of the 92 values are due to the umbrella variable being negative, as opposed to it being missing. So, we have to figure out how we might want to divide these variables (binarize, or just keep the binary umbrella variable)?